In [130]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
import time
import matplotlib.pyplot as plt
import re
import pickle
import tensorflow as tf

In [4]:
### Matrices
class matrix_expansion:
    
    def __init__(self,active_motors):
        self.active_motors = active_motors
        active_sensors = np.array(active_motors*2)
        active_sensors[len(active_motors):] += 14
        self.active_sensors = active_sensors
        self.shape = ()
        
    def load_from_file(self,filename):
        f = open(filename,"r")
        matrix = f.read()
        f.close()
        matrix = re.split(",NEW_ROW,",matrix)
        matrix.pop()
        matrix = np.array([np.array(re.split(",", row)).astype(np.float) for row in matrix])
        self.shape = matrix.shape
        return matrix
        
    def reduced_matrix(self,matrix):
        matrix = matrix[:,self.active_sensors][self.active_motors]
        return matrix
    
    def expanded_matrix(self,reduced_matrix):
        matrix = np.zeros(self.shape)
        flat = reduced_matrix.flatten()
        matrix = np.zeros((14,28))
        k = 0
        for i in active_motors:
            for j in active_sensors:
                matrix[i,j] = flat[k]
                k += 1
        return matrix

In [7]:
# Encoding

# HTM SDR Scalar Encoder
# Input: Scalar
# Parameters: n - number of units, w - bits used to represent signal (width), b - buckets (i.e. resolution), 
#             min - minimum value of input (inclusive), max - maximum input value (inclusive)
class scalar_sdr:
    
    def __init__(self, b, w, min_, max_, shape=(1,1), neg=True):
        if type(b) != int or type(w) != int or type(min_) != float or type(max_) != float:
            raise TypeError("b - buckets must be int, w - width must be int, min_ must be float and max_ must be float")
        self.b = b # must be int
        self.w = w # must be int
        self.min = min_ # must be float
        self.max = max_ # must be float
        self.n = b+w-1 # number of units for encoding
        self.ndarray_shape = shape
        self.nodes = self.n*reduce(lambda x, y: x*y, self.ndarray_shape)
        self.neg = neg
        
    def encode(self,input_):
        if input_ > self.max or input_ < self.min:
            raise ValueError("Input outside encoder range!")
        if type(input_) != float:
            raise TypeError("Input must be float!")
        if self.neg:
            output = np.zeros(self.n)-1
        else:
            output = np.zeros(self.n)
        index = int((input_-self.min)/(self.max-self.min)*self.b)
        output[index:index+self.w] = 1
        return output
    
    def encode_ndarray(self,input_):
        if input_.shape != self.ndarray_shape:
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        output = []
        for i in np.nditer(input_, order='K'):
            output.append(self.encode(float(i)))
        return np.array(output)
    '''
    def decode(self,input_):
        if len(input_) != self.n: # or len(np.nonzero(input_+1)[0]) != self.w: <-- Can't have since the network is not guaranteed to produce this by any means!!!
            raise TypeError("Input does not correspond to encoder encoded data!")
        # output = np.nonzero(input_+1)[0][0]/float(self.b)*(self.max-self.min)+self.min <-- This doesn't work really since bits can randomly fire, taking the average is a more reasonable decoding
        median = np.median(np.nonzero(input_+1)[0])            
        try:
            output = int(median-float(self.w)/2.0)/float(self.b)*(self.max-self.min)+self.min # i.e. figure out center (median more outlier resistant than mean) and subtract width/2
        except ValueError:
            output = None
        return output
    '''
    def decode(self,input_):
        if len(input_) != self.n: 
            raise TypeError("Input length does not match encoder length!")
        if len(np.nonzero(input_+1)[0]) == 0:
            return np.nan
        max_ = 0
        output = 0.0
        for i in range(self.b):
            x = np.zeros(self.n)-1
            x[i:i+self.w] = 1
            if x.shape != input_.shape:
                input_ = input_.reshape(x.shape)
            score = np.inner(x,input_) # this was broken for some input formats 
            if score > max_:
                max_ = score
                output = float(i)/float(self.b)*(self.max-self.min)+self.min
        return output
            
    def decode_ndarray(self,input_):
        if input_.shape != (reduce(lambda x, y: x*y, self.ndarray_shape)*self.n,): 
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        input_ = input_.reshape(self.ndarray_shape+(self.n,))
        output = []
        for i in np.ndindex(self.ndarray_shape):
            output.append(self.decode(input_[i]))
        output = np.array(output).reshape(self.ndarray_shape)
        return output
    
    def set_ndarray_shape(self,shape):
        if type(shape) != tuple:
            raise TypeError("Must provide tuple of array dimensions!")
        self.ndarray_shape = shape

In [21]:
#LAM

class LAM:
    def __init__(self,shape,weights=None):
        self.shape = shape
        try:
            if weights == None:
                self.weights = np.zeros(shape)
        except ValueError:
            self.weights = weights
    
    def store(self,input,output):
        dW = np.outer(input,output)
        self.weights += dW
        
    def recall(self,input,threshold=0,print_=False):
        output = input.dot(self.weights)-threshold
        if print_ == True:
            print output
        output[output > 0] = 1
        output[output < 0] = 0
        return output

    def save_weights(self,filename):
        np.save(filename, self.weights)
        
    def load_weights(self,filename):
        weights = np.load(filename)
        if weights.shape == self.shape:
            self.weights = weights
        else:
            raise ValueError("Dimensions of specified weight array does not match network weight dimensions!")

In [117]:
# Let's first generate a current state of the behaviors stored in the brain, the state of the encoder being used
# and the state of the memory networks

# Brain encoder
brain_encoder = scalar_sdr(1000,21,0.,1000.,neg=False)

# Behaviors stored in brain and ids associated with them
names = ["zero","fb","fs","sd"]
# Store the brain ids with no overlap, but with a fair amount of width.
# This results in perfect memory recall and noise robustness.
# The draw back is that essentially each id has its own isolated inputs and connections, 
# while sharing a common output with the other ids. The encoder in essence consequently serves as
# a selection mechanism, which is quite questionable in terms of biological plausibility.
brain_ids = [float(i)*brain_encoder.w for i in range(len(names))]
behaviors = dict(zip(names,brain_ids))

In [134]:
# Matrix Data

active_motors = [1,3,4,5,10,12]
expander = matrix_expansion(active_motors)

# Front back
filename = "/home/markus/dep/dep_matrices/front_back.dep"
fb_matrix = expander.load_from_file(filename)
fb_reduced = expander.reduced_matrix(fb_matrix)
#fb_expanded = expander.expanded_matrix(fb_reduced)

# Front side
filename = "/home/markus/dep/dep_matrices/front_side.dep"
fs_matrix = expander.load_from_file(filename)
fs_reduced = expander.reduced_matrix(fs_matrix)
#fs_expanded = expander.expanded_matrix(fs_reduced)

# Side down
filename = "/home/markus/dep/dep_matrices/side_down.dep"
sd_matrix = expander.load_from_file(filename)
sd_reduced = expander.reduced_matrix(sd_matrix)
#sd_expanded = expander.expanded_matrix(sd_reduced)

# Zero
zero_matrix = np.zeros(fb_matrix.shape)
zero_reduced = np.zeros(fb_reduced.shape)

matrices = {"fb": fb_matrix, "fs": fs_matrix, "sd": sd_matrix, "zero": zero_matrix}
#matrices = {"fb": fb_reduced, "fs": fs_reduced, "sd": sd_reduced, "zero": zero_reduced}

In [119]:
# Brain id to matrix memory network

active_motors = [1,3,4,5,10,12]
n = len(active_motors)
matrix_shape = (n,n*2)
matrix_encoder = scalar_sdr(100,21,-0.25,0.25,matrix_shape,neg=False)
shape = (brain_encoder.n,matrix_encoder.n*reduce(lambda x, y: x*y, m_encoder.ndarray_shape))

matrix_memories = LAM(shape)
for id_ in behaviors:    
    brain_sig = brain_encoder.encode(behaviors[id_])
    matrix = m_encoder.encode_ndarray(matrices[id_])
    matrix_memories.store(brain_sig,matrix)

In [131]:
# need function that does DEP calculation from input to output

with tf.name_scope("input"):
    input_ = tf.placeholder(tf.float32, [None,28], name="input") #differential sensors
with tf.name_scope("weights"):
    weights = tf.placeholder(tf.float32, [None,14,28], name="weights")
with tf.name_scope("output"):
    in_ = tf.reduce_sum(weights*input_,axis=2)
    out_ = tf.tanh(in_)
    #out_ = tf.sigmoid(in_)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

def dep_out(sensors, matrix):
    out = sess.run(out_,{input_: sensors, weights: matrix})
    return out

In [156]:
# Lets use the sine sweep behavior representation to obtain the behavior pattern
class behavior_from_matrix:

    def __init__(self,matrix):
        self.matrix = matrix
        self.pattern = self.getPattern()
        
    def getPattern(self):
        steps = 100
        sines = [np.sin(np.arange(steps)*2*np.pi/steps+i*2*np.pi/28) for i in range(28)]
        data = np.array([dep_out(sensors.reshape(1,28),self.matrix.reshape(1,14,28)).reshape(14,) for sensors in np.array(sines).T])
        return data

    def closest_phase(self,state):
        RMS = [np.sqrt(np.sum((self.pattern[i]-state)**2))for i in range(len(self.pattern))]
        indices_sorted = np.argsort(RMS)
        return -indices_sorted[0]
    
    def delayed_sensors(self,index,delay=50):
        a = np.roll(self.pattern,index,axis=0)
        return a[-delay:]

In [157]:
fb = pickle.load(open("/home/markus/dep/dep_data/bases/fb.pickle","rb"))
fs = pickle.load(open("/home/markus/dep/dep_data/bases/fs.pickle","rb"))
sd = pickle.load(open("/home/markus/dep/dep_data/bases/sd.pickle", "rb"))
zero = [np.zeros(fb[0].shape),np.zeros(fb[1].shape), np.zeros(fb[2].shape)]

bases = {"fb": fb, "fs": fs, "sd": sd, "zero": zero}
ts = {'fb_to_fs': 126, 'fb_to_sd': 126, ('fs','fb'): 128, , 'sd': 121, 'zero': 0}
transitions = dict(bases.keys(),[])

In [162]:
weights = dict(zip(behaviors.keys(),[{"weights_pos":[], "weights_vel":[]} for i in behaviors]))

# Generate weights
pos_encoder = scalar_sdr(41,3,-100000.0,100000.0,shape=(6,1),neg=False)
vel_encoder = scalar_sdr(3,1,-70.0,70.0,shape=(6,1),neg=False)

for id_ in weights:
    for i in active_motors:
        pos = pos_encoder.encode(float(bases[id_][0][ts[id_]][i])) # this gets the transition point motor status and encodes it
        m_pos = pos # hebbian learning of weights with only direct connections
        weights[id_]["weights_pos"].append(m_pos)
        
        vel = vel_encoder.encode(float(bases[id_][1][ts[id_]][i]))
        m_vel = np.outer(pos,vel) # hebbian learning of weights
        weights[id_]["weights_vel"].append(m_vel)

weights[id_]["weights_pos"] = np.array(behvs[id_]["weights_pos"]).reshape(1,6,pos_encoder.n,1)
weights[id_]["weights_vel"] = np.array(behvs[id_]["weights_vel"]).reshape(1,6,pos_encoder.n,vel_encoder.n)
weights["pos_encoder"] = pos_encoder
weights["vel_encoder"] = vel_encoder

In [100]:
# Let's add a new id in the brain
names.append("mix")
brain_ids.append(float((brain_id[-1]+1))*brain_encoder.w)
behaviors = dict(zip(names,brain_ids))

In [20]:
# Now we need to update the existing brain_id to dep_matrix memory network